In [1]:
import importlib
import sys
import random
from urllib.request import urlopen
from urllib.request import Request

from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal, LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfinterp import PDFTextExtractionNotAllowed
from pdfminer.pdfparser import PDFParser, PDFDocument

In [14]:
def parse(_path,text):
    with open(_path,'rb')as fp:# rb以二进制读模式打开本地pdf文件

        # 用文件对象来创建一个pdf文档分析器
        praser_pdf = PDFParser(fp)

        # 创建一个PDF文档
        doc = PDFDocument()

        # 连接分析器 与文档对象
        praser_pdf.set_document(doc)
        doc.set_parser(praser_pdf)

        # 提供初始化密码doc.initialize("123456")
        # 如果没有密码 就创建一个空的字符串
        doc.initialize()

        # 检测文档是否提供txt转换，不提供就忽略
        if not doc.is_extractable:
            raise PDFTextExtractionNotAllowed
        else:
            # 创建PDf资源管理器 来管理共享资源
            rsrcmgr = PDFResourceManager()

            # 创建一个PDF参数分析器  创建一个PDF设备对象
            laparams = LAParams()

            # 创建聚合器
            device = PDFPageAggregator(rsrcmgr, laparams=laparams)

            # 创建一个PDF页面解释器对象
            interpreter = PDFPageInterpreter(rsrcmgr, device)

            # 循环遍历列表，每次处理一页的内容
            # doc.get_pages() 获取page列表
            for page in doc.get_pages():
                # 使用页面解释器来读取
                interpreter.process_page(page)

                # 使用聚合器获取内容
                layout = device.get_result()

                # 这里layout是一个LTPage对象 里面存放着 这个page解析出的各种对象 一般包括LTTextBox, LTFigure, LTImage, LTTextBoxHorizontal 等等 想要获取文本就获得对象的text属性，
                for out in layout:
                    # 判断是否含有get_text()方法，图片之类的就没有
                    # if hasattr(out,"get_text"):
                    if isinstance(out, LTTextBoxHorizontal):
                        results = out.get_text()
                        text.write(results)


In [18]:
_path = r"Fintech100-2018-Report_Final_22-11-18sm.pdf"

In [15]:
with open('text.txt','a',encoding='utf-8') as text:
    parse(_path,text)

## 截取特定的pdf页

In [21]:
from PyPDF2 import PdfFileWriter, PdfFileReader

In [22]:
def  pdfCrap(path,start_page,i,save_path):
    """
    从pdf文件中截取几页，并保存在对应pdf文件中
    """
    # 开始页
    start_page = start_page - 1
    # 截止页
    end_page = start_page +  i  # 这里设定截取几页
    output = PdfFileWriter()
    pdf_file = PdfFileReader(open(path, "rb"), strict=False)
    pdf_pages_len = pdf_file.getNumPages()
    for i  in  range(start_page, end_page):
        output.addPage(pdf_file.getPage(i)) # 在输出流中添加页
    outputStream =  open(save_path, "wb")
    output.write(outputStream)

In [23]:
save_path = r"F:\project\ML_project\Fintech100-10-11.pdf"
pdfCrap(_path,10,2,save_path)

## PyPDF2不适合读取

In [24]:
def getPdfContent(filename):
    pdf = PdfFileReader(open(filename, "rb"))
    content = ""
    for i in range(0, pdf.getNumPages()):
        pageObj = pdf.getPage(i)

        extractedText = pageObj.extractText()
        content += extractedText + "\n"
        # return content.encode("ascii", "ignore")
    return content

In [25]:
getPdfContent(save_path)

'10\npage#0112#02JD Finance\n13#03\n14#0415#05\n16#0617#0718#08\n19#09\n20#10\n21#11\n\n22#12\n23#13\n24#14\n25#15\n26#1627#17\n28#18\n29#1930#20\n \n31#21\n32#22\n33#23\n34#2435#25\n36#26\n37#27\n38#28\n39#2940#30\n41#3142#32\n43#33\n44#34\n45#35\n46#3647#37\n48#38\n49#3950#40\n51#41\n52#4253#43\n54#44Folio55#4556#4657#47\n58#48\n59#49\n60#5061\n11\npage\n62\n63\n64\n65\n6667\n68697071\n72\n73\n74\n75\n7677\n78\n79\n80\n81\n82\n83848586\n87\n88\n899091\n92\n9394\n95\n96\n97\n98\n99Tala\n100\n101TenX\n102\n103\n104\n105106\n107\n108\n109\n110\n111presented in alphabetical order\n\n˜  \n  \n\n'

## 用定义的parse函数来解析pdf

In [27]:
with open('texe_10_11.txt','a',encoding='utf-8') as texe_10_11:
    parse(save_path,texe_10_11)